In [23]:
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import scipy.sparse as sparse
from re import split
import itertools
import os
import math
import openpyxl
import inspect
import xlsxwriter

In [24]:
# Python function to convert a list to string
    
# Function to convert  
def listToString(s): 
    
    # initialize an empty string
    str1 = " " 
    
    # return string  
    return (str1.join(s))

In [25]:
#b = [x for x in str_list if x != '']
def conList(str_list):
    str_list = str_list.split(' ')
    b = []
    for item in str_list:
        if item != '':
            if item.isdigit():
                b.append([int(item)])
            else:
                helplist = []
                listforrange = [int(elem) for elem in split('to|By',item)]
                for elem in range(*listforrange):
                    helplist.append(elem)
                b.append(helplist)
    flat_list = []
    for sublist in b:
        for item in sublist:
            flat_list.append(item)
    return flat_list

In [26]:
def printListtoRbt(listToRbt):
    listToPrint = ""
    for elem in listToRbt:
        listToPrint = listToPrint + " " +str(elem)
    return print(listToPrint)

In [27]:
def dfmaker(pathtocsv):
    df = pd.read_csv(os.getcwd() + '\\Data\\' + pathtocsv + '.csv', encoding = "utf-16", na_values='-', sep=";", decimal=",", low_memory=False).fillna(0)
    if 'bar' in pathtocsv.lower():
        barsdf = df.drop(df.index[len(df)-1])
        barsdf = barsdf.astype({"Bar": int, "Node 1": int, "Node 2": int})        
        return barsdf
    elif 'node' in pathtocsv.lower():
        nodesdf = df.drop(df.index[len(df)-1]).iloc[:, :-1].drop_duplicates(subset="Node").apply(pd.to_numeric)
        #forcesdf = pd.read_csv('Forces.csv', encoding = "utf-16", sep=';')
        return nodesdf
    elif 'force' in pathtocsv.lower():
        forcesdf = df.join(df['Bar/Node/Case'].str.split('/', expand=True).rename(columns={0:'Bar', 1:'Node', 2:'Case'})).iloc[:, 1:].apply(pd.to_numeric)
        forcesdf = forcesdf.astype({"Bar": int, "Node": int, "Case": int})
        return forcesdf
    else:
        return ValueError   
        

In [28]:
def barsdfmod(barsdf, nodesdf):
    #Modification of barsdf, add "direction"
    barsdf = barsdf.merge(nodesdf, left_on='Node 1', right_on='Node')
    barsdf = barsdf.merge(nodesdf, left_on='Node 2', right_on='Node')
    barsdf['deltaX'] = barsdf['X (m)_x'] - barsdf['X (m)_y']
    barsdf['deltaY'] = barsdf['Y (m)_x'] - barsdf['Y (m)_y']
    #barsdf['deltaZ'] = barsdf['Z (m)_x']-barsdf['Z (m)_y']
    #barsdf['signX'] = np.sign(barsdf['deltaX'])
    #barsdf['signY'] = np.sign(barsdf['deltaY'])
    barsdf['Direction'] = barsdf.apply(lambda r: tuple(r)[-2:], axis=1).apply(np.array)
    #barsdf['Angle'] = np.degrees(np.arctan(barsdf['deltaX'] / barsdf['deltaY']))
    barsdf = barsdf[['Bar', 'Node 1', 'Node 2', 'Direction', 'Section']]
    return barsdf

In [29]:
def extremum(listofelem, maxminabs):
    if maxminabs == 'max':
        return max(listofelem)
    elif maxminabs == 'min':
        return min(listofelem)
    else:
        return abs(listofelem)

In [30]:
def nodeinverse(barsdf, Node):
    df = pd.concat([barsdf[barsdf['Node 1'] == Node], barsdf[barsdf['Node 2'] == Node]])
    Node2 = []
    for i in range(len(df)):
        if df['Node 1'].tolist()[i]!=Node:
            Node2.append(df['Node 1'].tolist()[i])
            df['Direction'].iloc[i] = -df['Direction'].iloc[i]
            #df['Angle'].iloc[i] = -df['Angle'].iloc[i]
        else:
            Node2.append(df['Node 2'].tolist()[i])
    Node1 = [Node for i in range(len(df))]
    df['Node 1'] = Node1
    df['Node 2'] = Node2
    return df

In [31]:
def extremalforces(barsdf, forcesdf, listofnodes, forcetype, maxminabs, section=True):
    
    forcestypedict = {
      "FX": "FX (kN)",
      "FY": "FY (kN)",
      "FZ": "FZ (kN)",
      "MX": "MX (kNm)",
      "MY": "MY (kNm)",
      "MZ": "MZ (kNm)"
    }   

    Forcemin = True
    Forcemax = True

    #zmiana kolejności Node'ow
    for Node in listofnodes:
        df = nodeinverse(barsdf, Node)
        selforcesdf = forcesdf[forcesdf['Bar'].isin(df['Bar'].tolist())]
        selforcesdf = selforcesdf[selforcesdf['Node'] == Node]
        df = df.merge(selforcesdf, on='Bar', how='right')
        df.drop('Node', inplace=True, axis=1)
        if type(section) is not str:
            selcolumn = df[forcestypedict[forcetype]]
        else:
            if section in list(df['Section']):
                selcolumn = df[df['Section'] == section][forcestypedict[forcetype]]
            else:
                selcolumn = math.sqrt(-1) #ErrorValue
        
        if Forcemin:
            Forcemin = selcolumn.min()
        if Forcemax:
            Forcemin = selcolumn.max()
        
        if maxminabs == 'min':
            if (selcolumn.min() <= Forcemin):
                dfout = df
                idx = selcolumn.idxmin()
                Forcemin = selcolumn.min()
                dfout = dfout[dfout['Case'] == dfout.loc[idx]['Case']]
        elif maxminabs == 'max':
            if (selcolumn.max() >= Forcemax):
                dfout = df
                Forcemax = selcolumn.max()
                idx = selcolumn.idxmax()
                dfout = dfout[dfout['Case'] == dfout.loc[idx]['Case']]
        elif maxminabs == 'abs':
            if (abs(selcolumn).max() >= Forcemax):
                dfout = df
                idx = abs(selcolumn).idxmax()
                Forcemax = abs(selcolumn).max()
                dfout = dfout[dfout['Case'] == dfout.loc[idx]['Case']]
    return dfout

In [32]:
def extremalforcesall(inputdict, args):
    try:
        forcetype, maxminabs, section = args
    except:
        forcetype, maxminabs = args
        section = True
    
    forcestypedict = {
      "FX": "FX (kN)",
      "FY": "FY (kN)",
      "FZ": "FZ (kN)",
      "MX": "MX (kNm)",
      "MY": "MY (kNm)",
      "MZ": "MZ (kNm)"
    }  
    maxvalue = 0
    #dfout = math.sqrt(-1) ErrorValue
    for key in inputdict:
        try:
            df = extremalforces(inputdict[key][1][0], inputdict[key][1][2], inputdict[key][0], forcetype, maxminabs, section)         
            if type(section) is not str:
                maxcurforce = abs(df[forcestypedict[forcetype]]).max()
            else:
                if section in list(df['Section']):
                    maxcurforce = abs(df[df['Section'] == section][forcestypedict[forcetype]]).max()
                else:
                    maxcurforce = math.sqrt(-1) #ErrorValue

            if maxcurforce > maxvalue: 
                maxvalue = maxcurforce
                dfout = df
        except:
            None
    return dfout    

In [41]:
# conno = 'CON4'
# conList_p1 = ' 179 420 423 426 429 433 436 438 441 445 447 369 475 405 385 481 130 8 119 140 149 156 176 180 185 193 105 12 520 393 507 517 379 190 ' 
# conList_p2 = ' 1 522 1160to1168 1170 1171 1175 1177to1183 1185to1187 1192 1193 1195 1197   1199to1201 1203 1205 1206 1210 1216 1225 1227 '
# conList_p3 = ' 1536 1552to1559 1561to1565 1567to1571 1573to1580 1585 1588 1591 '

In [42]:
# conno = 'CON5a'
# conList_p1 = ' 223to259By12 233 243 251 255 275 277 279 313to321By2 329to335By2 346to352By2 ' 
# conList_p2 = ' 1239 1241 1252 1255 1259 1263 1277to1283By2 1288to1304By2 1341  '
# conList_p3 = ' 1602 1605to1625By4 1611 1619 1642to1664By2'

In [34]:
# conno = 'CON5'
# conList_p1 = ' 21 22 27 33 45 54 59  '
# conList_p2 = ' 1030 1049 1066'
# conList_p3 = ' '

In [61]:
conno = 'CON6'
conList_p1 = '  '
conList_p2 = ' 1005 1035 1054 1350 '
conList_p3 = ' 1385 1441 1449 '

In [51]:
# conno = 'CON6a'
# conList_p1 = ' 6 25 29 181 184 191 194 209  '
# conList_p2 = ' 1071 1080 1082 '
# conList_p3 = ' 1384 1385 1463 1478 1508  '

In [35]:
# conno = 'CON7'
# conList_p1 = '  '
# conList_p2 = ' 1054 '
# conList_p3 = ' 1449 1463 '

In [36]:
# conno = 'CON8'
# conList_p1 = ' '
# conList_p2 = ' 1054 '
# conList_p3 = ' 1677 '

In [62]:
conList_p1 = conList(conList_p1)
conList_p2 = conList(conList_p2)
conList_p3 = conList(conList_p3)

inputdict = {
    'p1' : [conList_p1, [dfmaker('Barsp1'), dfmaker('Nodesp1'), dfmaker('Forcesp1')]],
    'p2' : [conList_p2, [dfmaker('Barsp2'), dfmaker('Nodesp2'), dfmaker('Forcesp2')]],
    'p3' : [conList_p3, [dfmaker('Barsp3'), dfmaker('Nodesp3'), dfmaker('Forcesp3')]]
}
for key in inputdict:
    inputdict[key][1][0] = barsdfmod(inputdict[key][1][0], inputdict[key][1][1])

In [66]:
listofcases = [['FZ', 'abs'],['FZ', 'abs', 'IPE 160'],['MY', 'abs'], ['FX', 'abs', 'PO 16']]

In [67]:
for elem in listofcases:
    display(extremalforcesall(inputdict, elem))

,Bar,Node 1,Node 2,Direction,Section,FX (kN),FY (kN),FZ (kN),MX (kNm),MY (kNm),MZ (kNm),Case
1052,79,1350,1037,"[0.0009999999999976694, 2.0009999999999994]",RP 250x150x4,0.31,0.01,-36.77,0.02,-77.40,0.02,1067
2337,80,1350,1034,"[0.0009999999999976694, -1.9989999999999997]",RP 250x150x6,0.04,0.02,47.58,-0.02,-77.41,0.02,1067
3622,363,1350,1035,"[4.000999999999998, 0.0009999999999994458]",IPE 160,-2.54,0.00,-11.51,0.00,0.00,-0.00,1067
4907,380,1350,1054,"[-3.9990000000000023, 0.0009999999999994458]",IPE 160,-2.77,-0.00,10.99,-0.01,0.00,-0.00,1067
6192,715,1350,1249,"[0.0009999999999976694, 0.0009999999999994458]",RO 127x4,106.89,-0.00,0.00,0.00,0.01,-0.00,1067
7477,1001,1350,522,"[2.0009999999999977, 2.0009999999999994]",PO 16,-0.37,0.00,0.00,0.00,0.00,0.00,1067
8762,1080,1350,1175,"[-1.980000000000004, 2.0009999999999994]",PO 16,0.00,0.00,0.00,0.00,0.00,0.00,1067


,Bar,Node 1,Node 2,Direction,Section,FX (kN),FY (kN),FZ (kN),MX (kNm),MY (kNm),MZ (kNm),Case
1049,79,1350,1037,"[0.0009999999999976694, 2.0009999999999994]",RP 250x150x4,0.31,-0.00,-36.37,0.02,-75.00,0.03,1064
2334,80,1350,1034,"[0.0009999999999976694, -1.9989999999999997]",RP 250x150x6,0.05,0.02,46.42,-0.02,-75.00,0.03,1064
3619,363,1350,1035,"[4.000999999999998, 0.0009999999999994458]",IPE 160,-2.40,-0.00,-11.51,-0.00,0.00,0.00,1064
4904,380,1350,1054,"[-3.9990000000000023, 0.0009999999999994458]",IPE 160,-2.62,0.00,10.99,-0.01,0.00,0.00,1064
6189,715,1350,1249,"[0.0009999999999976694, 0.0009999999999994458]",RO 127x4,105.33,0.00,0.00,0.00,0.01,0.00,1064
7474,1001,1350,522,"[2.0009999999999977, 2.0009999999999994]",PO 16,-0.37,0.00,0.00,0.00,0.00,0.00,1064
8759,1080,1350,1175,"[-1.980000000000004, 2.0009999999999994]",PO 16,0.00,0.00,0.00,0.00,0.00,0.00,1064


,Bar,Node 1,Node 2,Direction,Section,FX (kN),FY (kN),FZ (kN),MX (kNm),MY (kNm),MZ (kNm),Case
1089,175,1441,1442,"[-0.0, 2.0]",RP 250x150x6,0.22,-0.03,-45.93,0.01,-85.50,0.06,1104
2374,176,1441,1440,"[0.0, -2.0]",RP 250x150x6,0.30,0.03,40.97,0.00,-85.49,0.06,1104
3659,360,1441,1623,"[-2.3500000000000014, 0.0]",IPE 330,0.63,0.00,-45.20,0.00,0.00,-0.00,1104
4944,418,1441,1426,"[4.0, -0.0]",IPE 160,0.49,-0.00,-11.51,-0.01,0.00,0.00,1104
6229,733,1441,1600,"[-0.0, -0.0]",RO 127x4,53.26,0.00,-0.00,-0.00,-0.01,-0.00,1104
7514,1093,1441,1557,"[2.0, -2.0]",PO 16,0.00,0.00,0.00,0.00,0.00,0.00,1104
8799,1095,1441,1558,"[-2.0, -2.0]",PO 16,-0.11,0.00,0.00,0.00,0.00,0.00,1104


,Bar,Node 1,Node 2,Direction,Section,FX (kN),FY (kN),FZ (kN),MX (kNm),MY (kNm),MZ (kNm),Case
685,168,1449,1450,"[-0.0, 2.0]",RP 250x150x6,-3.44,-1.07,-31.70,0.00,-41.2,1.58,700
1970,169,1449,1448,"[0.0, -2.0]",RP 250x150x6,42.96,1.09,14.14,0.00,-41.2,1.67,700
3255,301,1449,1463,"[-4.0, 0.0]",IPE 160,-11.44,0.60,5.86,-0.00,-0.0,0.00,700
4540,424,1449,1434,"[4.0, -0.0]",IPE 160,10.02,-0.60,-5.86,0.00,-0.0,0.00,700
5825,731,1449,1678,"[-0.0, -0.0]",RO 127x16,57.61,82.47,0.00,0.09,0.0,-0.00,700
7110,1040,1449,1577,"[2.0, 2.0]",PO 16,-41.37,0.00,0.00,0.00,0.0,0.00,700
8395,1046,1449,1578,"[-2.0, 2.0]",PO 16,-7.95,0.00,0.00,0.00,0.0,0.00,700


In [68]:
with pd.ExcelWriter(os.getcwd() + '\\Results\\' + conno + '.xlsx', engine='xlsxwriter') as writer:
    for elem in listofcases:
        df = extremalforcesall(inputdict, elem)
        df.to_excel(writer, sheet_name = listToString(elem))